In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
)

import torch.nn as nn

def has_sentences(example):
    """
    Функция возвращает True, если пример содержит хотя бы одно предложение
    в documents_sentences (и, опционально, в response_sentences).
    Иначе возвращает False.
    """

    if "documents_sentences" not in example:
        return False
    if not isinstance(example["documents_sentences"], list):
        return False
    total_doc_sentences = 0
    for doc in example["documents_sentences"]:
        if isinstance(doc, list):
            total_doc_sentences += len(doc)
    if total_doc_sentences == 0:
        return False

    if "response_sentences" in example:
        if not isinstance(example["response_sentences"], list):
            return False
        if len(example["response_sentences"]) == 0:
            return False

    return True

MODEL_NAME = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def preprocess(example, max_length=1024):
    question_ids = tokenizer.encode(example["question"], add_special_tokens=False)
    doc_ids, rel_labels, util_labels = [], [], []

    for doc in example["documents_sentences"]:
        for key, sent in doc:
            tokens = tokenizer.encode(sent, add_special_tokens=False)
            doc_ids += tokens
            rel_labels += [float(key in example["all_relevant_sentence_keys"])] * len(tokens)
            util_labels += [float(key in example["all_utilized_sentence_keys"])] * len(tokens)

    response_ids = tokenizer.encode(example["response"], add_special_tokens=False)
    adh_labels = [float(example["adherence_score"])] * len(response_ids)

    sep_id = tokenizer.sep_token_id
    input_ids = question_ids + [sep_id] + doc_ids + [sep_id] + response_ids

    context_mask = [0] * (len(question_ids)+1) + [1]*len(doc_ids) + [0] + [0]*len(response_ids)
    response_mask = [0]*(len(question_ids)+len(doc_ids)+2) + [1]*len(response_ids)

    rel_labels = [0.0]*(len(question_ids)+1) + rel_labels + [0.0]*(len(response_ids)+1)
    util_labels = [0.0]*(len(question_ids)+1) + util_labels + [0.0]*(len(response_ids)+1)
    adh_labels = [0.0]*(len(question_ids)+len(doc_ids)+2) + adh_labels

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        context_mask = context_mask[:max_length]
        response_mask = response_mask[:max_length]
        rel_labels = rel_labels[:max_length]
        util_labels = util_labels[:max_length]
        adh_labels = adh_labels[:max_length]

    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor([1]*len(input_ids), dtype=torch.long),
        'context_mask': torch.tensor(context_mask, dtype=torch.bool),
        'response_mask': torch.tensor(response_mask, dtype=torch.bool),
        'labels_relevance': torch.tensor(rel_labels, dtype=torch.float),
        'labels_utilization': torch.tensor(util_labels, dtype=torch.float),
        'labels_adherence': torch.tensor(adh_labels, dtype=torch.float),
    }





/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from promt_test.data import load_full_ragbench

In [ ]:
dataset = load_full_ragbench()#load_dataset("rungalileo/ragbench", "delucionqa")
dataset = {key:dataset[key].filter(has_sentences) for key in dataset}

In [ ]:
max_len = tokenizer.model_max_length  # максимально допустимая длина входа

In [ ]:
def count_tokens(example):
    question_ids = tokenizer.encode(example["question"], add_special_tokens=False)
    doc_ids = []
    for doc in example["documents_sentences"]:
        for key, sent in doc:
            doc_ids.extend(tokenizer.encode(sent, add_special_tokens=False))
    response_ids = tokenizer.encode(example["response"], add_special_tokens=False)
    total = len(question_ids) + 1 + len(doc_ids) + 1 + len(response_ids)
    return total

def filter_by_length(example):
    return count_tokens(example) <= max_len


filtered = {
    ds_name: ds_dict.filter(filter_by_length)
    for ds_name, ds_dict in dataset.items()
}


In [ ]:

train_splits = [ds["train"] for ds in filtered.values() if "train" in ds]
val_splits   = [ds["validation"] for ds in filtered.values() if "validation" in ds]
test_splits  = [ds["test"] for ds in filtered.values() if "test" in ds]

In [ ]:
from datasets import DatasetDict, concatenate_datasets
combined = DatasetDict({
    "train":      concatenate_datasets(train_splits),
    "validation": concatenate_datasets(val_splits),
    "test":       concatenate_datasets(test_splits)
})
combined

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # batch: list of dicts
    keys = batch[0].keys()
    out = {}
    #  max len
    max_len = max_len #max(len(x['input_ids']) for x in batch)
#     print(batch)
    for k in keys:
        vals = [x[k] for x in batch]
        if k == 'input_ids':
            out[k] = pad_sequence(vals, batch_first=True, padding_value=tokenizer.pad_token_id)
        elif k == 'attention_mask':
            out[k] = pad_sequence(vals, batch_first=True, padding_value=0)
        elif k in ['context_mask', 'response_mask']:
            out[k] = pad_sequence(vals, batch_first=True, padding_value=False)
        elif k.startswith('labels_'):
            out[k] = pad_sequence(vals, batch_first=True, padding_value=0)
        else:
            out[k] = torch.stack(vals)
    return out
train_data = [preprocess(ex) for ex in combined['train']]
val_data   = [preprocess(ex) for ex in combined['validation']]
batch_size=8

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


In [ ]:
combined['train'][1].keys()

In [ ]:
import numpy as np

In [ ]:
np.mean(combined['validation']['relevance_score']), np.mean(combined['validation']['utilization_score']), np.mean(combined['validation']['completeness_score'])

In [ ]:
processed = preprocess(combined['train'][1])


tokens = tokenizer.convert_ids_to_tokens(processed["input_ids"])

labels_rel = processed["labels_relevance"]
labels_util = processed["labels_utilization"]
labels_adh = processed["labels_adherence"]

print(f"{'Token':20} | {'Relevance':9} | {'Utilization':11} | {'Adherence':9}")
print("-" * 60)
i = 0
for token, rel, util, adh in zip(tokens, labels_rel, labels_util, labels_adh):
    if token == '[PAD]':
        break
    i+=1
    print(f"{token:20} | {rel:9.1f} | {util:11.1f} | {adh:9.1f}")
    if token == '▁Simply':
        print(i)

In [17]:
import torch.nn.functional as F
from scipy.special import expit
import numpy as np
class DebertaTrace(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
        hid = base_model.config.hidden_size
        self.rel_head = nn.Linear(hid,1)
        self.util_head = nn.Linear(hid,1)
        self.adh_head = nn.Linear(hid,1)

    def forward(self, input_ids, attention_mask):
        out = self.base(input_ids=input_ids, attention_mask=attention_mask)
        hs = out.last_hidden_state
        return {
            'logits_relevance': self.rel_head(hs).squeeze(-1),
            'logits_utilization': self.util_head(hs).squeeze(-1),
            'logits_adherence': self.adh_head(hs).squeeze(-1)
        }


def compute_batch_metrics(logits, labels, masks):
    # logits: dict of tensors [B,L]
    # labels: dict of tensors [B,L]
    # masks: dict of bool tensors [B,L]
    rel_log, util_log, adh_log = logits['logits_relevance'], logits['logits_utilization'], logits['logits_adherence']
    rel_lab, util_lab, adh_lab = labels['labels_relevance'], labels['labels_utilization'], labels['labels_adherence']
    ctx_m, resp_m = masks['context_mask'], masks['response_mask']

    # probs
    rel_p = torch.tensor(expit(rel_log.detach().cpu().numpy()))
    util_p = torch.tensor(expit(util_log.detach().cpu().numpy()))
    adh_p = torch.tensor(expit(adh_log.detach().cpu().numpy()))
    # preds
    pred_rel = rel_p>0.5
    pred_util = util_p>0.5
    pred_adh = adh_p>0.5
    # true
    true_rel = rel_lab.detach().cpu()==1
    true_util = util_lab.detach().cpu()==1
    true_adh = adh_lab.detach().cpu()==1

    # per example
    def ex_metric(pred, true, mask):
        inter = (pred & true & mask).sum(dim=1).float()
        denom = (true & mask).sum(dim=1).float()
        return (inter/denom.clamp(min=1)).mean().item()

    rel_m = ex_metric(pred_rel, true_rel, ctx_m)
    util_m= ex_metric(pred_util, true_util, ctx_m)
    adh_m = ex_metric(pred_adh, true_adh, resp_m)
    comp = ((pred_rel & pred_util & ctx_m).sum(dim=1).float()/ (true_rel & ctx_m).sum(dim=1).float().clamp(min=1)).mean().item()
    return rel_m, util_m, adh_m, comp

In [18]:
def check_for_nans(logits, labels, mask, head_name=""):
    """
    Проверяет тензоры logits и labels для данного сегмента mask на наличие NaN.
    Если NaN найдены, выводит подробную информацию и выбрасывает ошибку.
    """
    # Извлечение элементов по маске
    logit_vals = logits[mask]
    label_vals = labels[mask]
    
    # Проверка на NaN
    nan_logits = torch.isnan(logit_vals)
    nan_labels = torch.isnan(label_vals)
    
    if nan_logits.any() or nan_labels.any():
        print(f"NaN detected in head '{head_name}':")
        if nan_logits.any():
            idxs = torch.nonzero(nan_logits, as_tuple=True)[0]
            print(f"  logits NaN at positions: {idxs.tolist()}")
            print(f"  sample logits: {logit_vals[idxs].tolist()}")
        if nan_labels.any():
            idxs = torch.nonzero(nan_labels, as_tuple=True)[0]
            print(f"  labels NaN at positions: {idxs.tolist()}")
            print(f"  sample labels: {label_vals[idxs].tolist()}")
        # Остановим обучение для отладки
        raise ValueError(f"NaN encountered in head '{head_name}'")

In [19]:
def compute_trace_metrics_inference(logits, masks, threshold=0.5):
    """
    Считает для каждого примера батча:
      - relevance_rate:   доля токенов контекста, предсказанных как релевантные
      - utilization_rate: доля токенов контекста, предсказанных как используемые
      - adherence_rate:   доля токенов ответа,   предсказанных как подкреплённые (adherent)
      - completeness:     доля релевантных предсказаний, которые ещё и используются:
                          |pred_rel ∧ pred_util| / |pred_rel|
    Параметры:
      logits – словарь из трёх [B,L] тензоров: logits_relevance,
               logits_utilization, logits_adherence
      masks  – словарь из двух [B,L] булевых тензоров: context_mask, response_mask
      threshold – граница для сигмоида, default=0.5
    Возвращает словарь из четырёх [B] тензоров с метриками по примерам.
    """
    rel_pred  = (torch.sigmoid(logits['logits_relevance'].detach().cpu())  > threshold)
    util_pred = (torch.sigmoid(logits['logits_utilization'].detach().cpu())> threshold)
    adh_pred  = (torch.sigmoid(logits['logits_adherence'].detach().cpu())   > threshold)

    ctx_m  = masks['context_mask'].detach().cpu()
    resp_m = masks['response_mask'].detach().cpu()

    def rate(pred, mask):
        # sum(pred & mask) / sum(mask)
        num = (pred & mask).sum(dim=1).float()
        den = mask.sum(dim=1).float().clamp(min=1)
        return num.div(den)

    relevance_rate   = rate(rel_pred,  ctx_m)
    utilization_rate = rate(util_pred, ctx_m)
    adherence_rate   = rate(adh_pred,  resp_m)

    # completeness: из релевантных предсказаний — сколько ещё и util
    num_ru = (rel_pred & util_pred & ctx_m).sum(dim=1).float()
    den_r  = rel_pred.sum(dim=1).float().clamp(min=1)
    completeness = num_ru.div(den_r)

    return {
        'relevance_rate':   relevance_rate,    
        'utilization_rate': utilization_rate,  
        'adherence_rate':   adherence_rate,
        'completeness':     completeness
    }

In [20]:
def compute_classification_metrics(logits, labels, masks):
    """
    Считает precision/recall/F1 по каждому TRACE-лейблу на всём батче.
    Возвращает словарь из девяти скалярных метрик.
    """
    rel_pred  = (torch.sigmoid(logits['logits_relevance'].detach().cpu())  > 0.5) & masks['context_mask']
    util_pred = (torch.sigmoid(logits['logits_utilization'].detach().cpu())> 0.5) & masks['context_mask']
    adh_pred  = (torch.sigmoid(logits['logits_adherence'].detach().cpu())   > 0.5) & masks['response_mask']

    rel_true  = (labels['labels_relevance']   == 1) & masks['context_mask']
    util_true = (labels['labels_utilization']== 1) & masks['context_mask']
    adh_true  = (labels['labels_adherence']   == 1) & masks['response_mask']

    def prf(pred, true):
        tp = (pred & true).sum().float()
        fp = (pred & ~true).sum().float()
        fn = (~pred & true).sum().float()
        prec = tp.div(tp + fp.clamp(min=1))
        rec  = tp.div(tp + fn.clamp(min=1))
        f1   = 2*prec*rec.div((prec+rec).clamp(min=1))
        return prec.item(), rec.item(), f1.item()

    rel_p,  rel_r,  rel_f1  = prf(rel_pred,  rel_true)
    util_p, util_r, util_f1 = prf(util_pred, util_true)
    adh_p,  adh_r,  adh_f1  = prf(adh_pred,  adh_true)

    return {
        'relevance_precision':    rel_p,
        'relevance_recall':       rel_r,
        'relevance_f1':           rel_f1,
        'utilization_precision':  util_p,
        'utilization_recall':     util_r,
        'utilization_f1':         util_f1,
        'adherence_precision':    adh_p,
        'adherence_recall':       adh_r,
        'adherence_f1':           adh_f1,
    }

In [21]:
val_metrics_per_examples = []

In [28]:
import torch

def compute_classification_metrics_stream(all_logits, all_labels, all_masks, threshold=0.5):
    """
    all_logits: dict с ключами 'logits_relevance', 'logits_utilization', 'logits_adherence'
                значениями — списки тензоров [B_i, L_i]
    all_labels: dict с ключами 'labels_relevance', 'labels_utilization', 'labels_adherence'
                значениями — списки тензоров [B_i, L_i]
    all_masks:  dict с ключами 'context_mask', 'response_mask'
                значениями — списки булевых тензоров [B_i, L_i]
    threshold: порог для сигмоиды
    
    Возвращает dict с precision/recall/F1 для каждой из трёх голов.
    """
    # Собираем по-masked позициям предсказания и ground truth в плоские булевы тензоры
    rel_pred_list, rel_true_list = [], []
    util_pred_list, util_true_list = [], []
    adh_pred_list, adh_true_list   = [], []

    for (
        lr, lu, la,
        tr, tu, ta,
        ctx_m, resp_m
    ) in zip(
        all_logits['logits_relevance'],
        all_logits['logits_utilization'],
        all_logits['logits_adherence'],
        all_labels['labels_relevance'],
        all_labels['labels_utilization'],
        all_labels['labels_adherence'],
        all_masks['context_mask'],
        all_masks['response_mask'],
    ):
        # бинаризуем предсказания
        p_rel  = (torch.sigmoid(lr) > threshold)
        p_util = (torch.sigmoid(lu) > threshold)
        p_adh  = (torch.sigmoid(la) > threshold)
        # истинные метки
        t_rel  = (tr == 1)
        t_util = (tu == 1)
        t_adh  = (ta == 1)

        # собираем только те позиции, где маска = True
        rel_pred_list.append( p_rel[ctx_m].cpu() )
        rel_true_list.append( t_rel[ctx_m].cpu() )
        util_pred_list.append( p_util[ctx_m].cpu() )
        util_true_list.append( t_util[ctx_m].cpu() )
        adh_pred_list.append( p_adh[resp_m].cpu() )
        adh_true_list.append( t_adh[resp_m].cpu() )

    # конкатенируем
    rel_pred_all  = torch.cat(rel_pred_list)
    rel_true_all  = torch.cat(rel_true_list)
    util_pred_all = torch.cat(util_pred_list)
    util_true_all = torch.cat(util_true_list)
    adh_pred_all  = torch.cat(adh_pred_list)
    adh_true_all  = torch.cat(adh_true_list)

    # теперь считаем TP/FP/FN
    def prf(pred, true):
        tp = int((pred & true).sum())
        fp = int((pred & ~true).sum())
        fn = int((~pred & true).sum())
        prec = tp / (tp + fp) if tp + fp > 0 else 0.0
        rec  = tp / (tp + fn) if tp + fn > 0 else 0.0
        f1   = (2 * prec * rec / (prec + rec)) if (prec + rec) > 0 else 0.0
        return prec, rec, f1

    rel_p, rel_r, rel_f1   = prf(rel_pred_all,  rel_true_all)
    util_p, util_r, util_f1 = prf(util_pred_all, util_true_all)
    adh_p, adh_r, adh_f1    = prf(adh_pred_all,  adh_true_all)

    return {
        'relevance_precision':    rel_p,
        'relevance_recall':       rel_r,
        'relevance_f1':           rel_f1,
        'utilization_precision':  util_p,
        'utilization_recall':     util_r,
        'utilization_f1':         util_f1,
        'adherence_precision':    adh_p,
        'adherence_recall':       adh_r,
        'adherence_f1':           adh_f1,
    }


In [29]:
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base = AutoModel.from_pretrained(MODEL_NAME)
model=DebertaTrace(base).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=2e-5)
scaler = torch.amp.GradScaler('cuda')
accum=4

for epoch in range(5):
    model.train()
    total_loss=0
    opt.zero_grad()
    loop = tqdm(train_loader, desc=f"Training Epoch {epoch}", total=len(train_loader))
    for i, batch in enumerate(loop, 1):
        inputs = {'input_ids': batch['input_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}
        labels = {k: v.to(device) for k,v in batch.items() if k.startswith('labels_')}
        masks  = {k: v.to(device) for k,v in batch.items() if k.endswith('_mask')}
        with torch.amp.autocast('cuda'):
            out = model(**inputs)
            
            l_rel = F.binary_cross_entropy_with_logits(out['logits_relevance'][masks['context_mask']], labels['labels_relevance'][masks['context_mask']])

            l_util= F.binary_cross_entropy_with_logits(out['logits_utilization'][masks['context_mask']], labels['labels_utilization'][masks['context_mask']])

            l_adh = F.binary_cross_entropy_with_logits(out['logits_adherence'][masks['response_mask']], labels['labels_adherence'][masks['response_mask']])
            loss = (l_rel + l_util + l_adh)/ 3 #/ accum
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#         if (i+1)%accum==0:
        scaler.step(opt)
        scaler.update()
        opt.zero_grad()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
        break
    print(f"Epoch {epoch} loss: {total_loss:.4f}")

    # валидация
    model.eval()
    sums = np.zeros(4);
    count=0
    metrics = {
    'relevance_rate':   [],  # доля токенов контекста, помеченных релевантными
    'utilization_rate': [],  # доля токенов контекста, помеченных используемыми
    'adherence_rate':   [],  # доля токенов ответа,   помеченных подкреплёнными
    'completeness':     []   # доля релевантных предсказаний, которые ещё и util
    }
    all_logits = {
        'logits_relevance':   [],
        'logits_utilization': [],
        'logits_adherence':   []
    }
    all_labels = {
        'labels_relevance':   [],
        'labels_utilization': [],
        'labels_adherence':   []
    }
    all_masks = {
        'context_mask':  [],
        'response_mask': []
    }
    with torch.no_grad():
        
        for batch in tqdm(val_loader):
            inputs = {'input_ids': batch['input_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}
            labels = {k: v for k,v in batch.items() if k.startswith('labels_')}
            masks  = {k: v for k,v in batch.items() if k.endswith('_mask')}
            out = model(**{k:inputs[k] for k in inputs})
            batch_metrics = compute_trace_metrics_inference(out, masks)
#             m = compute_batch_metrics(out, labels, masks)
            for name, values in batch_metrics.items():
                metrics[name].extend(values.cpu().tolist())
#             sums += np.array(m)
            for k in all_logits:
                all_logits[k].append(out[k].cpu())
            for k in all_labels:
                all_labels[k].append(labels[k].cpu())
            for k in all_masks:
                all_masks[k].append(masks[k].cpu())
    big_logits = { k: torch.cat(v, dim=0) for k, v in all_logits.items() }
    big_labels = { k: torch.cat(v, dim=0) for k, v in all_labels.items() }
    big_masks  = { k: torch.cat(v, dim=0) for k, v in all_masks.items() }
    class_metrics = compute_classification_metrics_stream(big_logits, big_labels, big_masks)
    print("\nClassification metrics on full validation set:")
    for name, val in class_metrics.items():
        print(f"  {name}: {val:.4f}")
#     avg = sums/count
    print(f"Eval -> relevance={np.mean(metrics['relevance_rate']):.4f}, utilization={np.mean(metrics['utilization_rate']):.4f}, adherence={np.mean(metrics['adherence_rate']):.4f}, completeness={np.mean(metrics['completeness']):.4f}")


Training Epoch 0:   0%|                                                                                                                                                           | 0/3371 [00:00<?, ?it/s, loss=0.668]


Epoch 0 loss: 0.6679


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 454/454 [00:45<00:00, 10.01it/s]



Classification metrics on full validation set:
  relevance_precision: 0.4492
  relevance_recall: 0.1319
  relevance_f1: 0.2040
  utilization_precision: 0.3367
  utilization_recall: 0.6322
  utilization_f1: 0.4394
  adherence_precision: 0.7805
  adherence_recall: 0.8504
  adherence_f1: 0.8139
Eval -> relevance=0.1151, utilization=0.5985, adherence=0.8578, completeness=0.1475


Training Epoch 1:   0%|                                                                                                                                                           | 0/3371 [00:00<?, ?it/s, loss=0.721]


...


Classification metrics on full validation set:
  relevance_precision: 0.4820
  relevance_recall: 0.2806
  relevance_f1: 0.3547
  utilization_precision: 0.3506
  utilization_recall: 0.5016
  utilization_f1: 0.4127
  adherence_precision: 0.7834
  adherence_recall: 0.9539
  adherence_f1: 0.8603
Eval -> relevance=0.2257, utilization=0.4540, adherence=0.9533, completeness=0.1597


Training Epoch 3:   0%|                                                                                                                                                           | 0/3371 [00:00<?, ?it/s, loss=0.635]


Epoch 3 loss: 0.6349


  1%|█▉                                                                                                                                                                                | 5/454 [00:00<00:56,  7.99it/s]


KeyboardInterrupt: 